In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
(I-sqrt(3)).minpoly()

x^4 - 4*x^2 + 16

In [3]:
var('nv')
K1.<i_minus_rad3> = NumberField(nv^4-4*nv^2+16)

In [4]:
i = i_minus_rad3^3/8
rad3 = (i_minus_rad3^3 - 8*i_minus_rad3)/8

In [5]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(K1)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [7]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i*rad3*c})

In [8]:
# Find simmetries and study possible subfamilies

In [9]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 4.42 s, sys: 96.8 ms, total: 4.51 s
Wall time: 41.1 s


In [10]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 5.08 s, sys: 100 ms, total: 5.18 s
Wall time: 7.03 s


In [11]:
%time conds = SE9.find_conditions_for_subfamilies(adm_SE9, simm_SE9)

CPU times: user 2min 11s, sys: 666 ms, total: 2min 12s
Wall time: 2min 14s


In [12]:
sols = [solve_linear_system(q, [f], [c]) for q in conds] 
for sol in sols:
    print(sol[0]//sol[1]*c)

(-1/4*i_minus_rad3^3 + i_minus_rad3 - 1)*c
(1/4*i_minus_rad3^3 - 1/2*i_minus_rad3^2 + 1)*c
(i_minus_rad3 + 1)*c
(-1/4*i_minus_rad3^3 - 1/2*i_minus_rad3^2 + 1)*c
(-i_minus_rad3 + 1)*c
(1/4*i_minus_rad3^3 - i_minus_rad3 - 1)*c


In [13]:
# Choose one subfamily (say SE9') and study it

In [14]:
SE9_1 = SE9.subs({f:(1/4*i_minus_rad3^3 - i_minus_rad3 - 1)*c})

In [15]:
%time adm_SE9_1 = SE9_1.find_admissible_projectivities()
len(adm_SE9_1)

CPU times: user 2.63 s, sys: 144 ms, total: 2.77 s
Wall time: 25.4 s


1296

In [16]:
simm_SE9_1 = SE9_1.find_simmetries(adm_SE9_1)
len(simm_SE9_1)

108

In [17]:
# Study how the simmetries permute the Eckardt points

In [18]:
Eck_points_perms = []
for proj in simm_SE9_1:
    Eck_points_perms.append(Permutation(SE9_1.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(36, '(C3 x C3) : C4')

In [19]:
# Study how the simmetries permute the 27 lines

In [20]:
lines_perms = []
for simm in simm_SE9_1:
    lines_perms.append(Permutation(SE9_1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(108, '((C3 x C3) : C3) : C4')

In [21]:
all_subgroups = lines_perms_group.subgroups()

In [22]:
# H is the subgroup of elements of order 3
H = [el for el in lines_perms_group.normal_subgroups() if el.order()==27][0]
list(set([el.order() for el in H if el.cycle_tuples() != []])) == [3], H.structure_description()

(True, '(C3 x C3) : C3')

In [23]:
# The center is C3
Z_H = H.center()
M = Z_H.gens_small()[0]
Z_H.structure_description(), M

('C3',
 (1,10,15)(2,18,9)(3,23,11)(4,12,26)(5,20,8)(6,7,17)(13,25,24)(14,19,27)(16,22,21))

In [24]:
H.gens()

[(1,4,6)(2,3,5)(7,10,12)(8,9,11)(13,22,27)(14,25,21)(15,26,17)(16,19,24)(18,23,20),
 (1,8,13)(2,16,4)(3,27,17)(5,25,10)(6,23,14)(7,11,19)(9,21,26)(12,18,22)(15,20,24),
 (1,10,15)(2,18,9)(3,23,11)(4,12,26)(5,20,8)(6,7,17)(13,25,24)(14,19,27)(16,22,21)]

In [25]:
H_over_Z_H = [el for el in all_subgroups if el.is_isomorphic(H.quotient(Z_H))][0]
[N, P] = H_over_Z_H.gens_small()
H_over_Z_H.structure_description(), N, P

('C3 x C3',
 (1,10,15)(2,18,9)(3,23,11)(4,12,26)(5,20,8)(6,7,17)(13,25,24)(14,19,27)(16,22,21),
 (1,17,12)(2,8,23)(3,9,20)(4,15,7)(5,11,18)(6,26,10)(13,19,21)(14,22,24)(16,25,27))

In [26]:
H.quotient(PermutationGroup([M, N])).is_isomorphic(PermutationGroup([P]))

False

In [27]:
C4 = [el for el in all_subgroups if el.is_isomorphic(lines_perms_group.quotient(H))][0]
B = C4.gens_small()[0]
B

(2,3,19,22)(4,20,7,24)(5,6,25,26)(8,17,13,12)(9,11,14,16)(18,23,27,21)

In [28]:
gens = H.gens()+[B]
gens_indices = [lines_perms.index(gen) for gen in gens]
matrix_gens = [simm_SE9_1[index] for index in gens_indices]

In [29]:
# There are twelve lines through 3 Eckardt points, and they are all coplanar

In [30]:
lines_three_eck = []
lines_labels = []
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            point1 = SE9_1.eckardt_points[i]
            point2 = SE9_1.eckardt_points[j]
            point3 = SE9_1.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                lines_labels.append([i+1,j+1,k+1])
                planes = get_two_planes_containing_line(point1, point2)
                lines_three_eck.append(Line(planes, [point1, point2], line12_pl))
print(lines_labels)

[[1, 2, 8], [1, 3, 9], [1, 4, 6], [1, 5, 7], [2, 3, 5], [2, 4, 9], [2, 6, 7], [3, 4, 7], [3, 6, 8], [4, 5, 8], [5, 6, 9], [7, 8, 9]]


In [31]:
False in [[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck]

False

In [32]:
lines_three_eck[0].get_plane_containing_another_incident_line(lines_three_eck[1])

x + (1/8*i_minus_rad3^2 - 1/2)*y + (-1/8*i_minus_rad3^2 + 1/2)*z

In [33]:
# To do, study how the 12 lines are permuted

In [34]:
for gen in matrix_gens:
    print(Permutation([lines_three_eck.index(line.apply_proj(gen))+1 for line in lines_three_eck]).to_permutation_group_element())

(1,8,11)(2,10,7)(4,6,9)
(1,11,8)(2,10,7)(3,5,12)
()
(1,2)(3,4)(5,9,12,6)(7,8,10,11)


In [35]:
[Eck_points_perms[gen_index] for gen_index in gens_indices]

[(1,6,4)(2,5,3)(7,8,9), (1,7,5)(2,4,9)(3,6,8), (), (2,9,8,3)(4,7,6,5)]

In [36]:
keys = list(SE9.cl_lines.keys())
print(SE9.eckardt_points_labels)
print()
for gen in gens:
    labels = from_perm_to_labels(gen)
    print([[keys[labels.index(label)] for label in Eck_label] for Eck_label in SE9.eckardt_points_labels])
    print()

[['E1', 'G4', 'F14'], ['E2', 'G3', 'F23'], ['E3', 'G5', 'F35'], ['E4', 'G6', 'F46'], ['E5', 'G2', 'F25'], ['E6', 'G1', 'F16'], ['F12', 'F36', 'F45'], ['F13', 'F24', 'F56'], ['F15', 'F26', 'F34']]

[['E6', 'G1', 'F16'], ['E5', 'G2', 'F25'], ['E2', 'G3', 'F23'], ['E1', 'G4', 'F14'], ['E3', 'G5', 'F35'], ['E4', 'G6', 'F46'], ['F56', 'F24', 'F13'], ['F26', 'F15', 'F34'], ['F36', 'F45', 'F12']]

[['F12', 'F45', 'F36'], ['E4', 'F46', 'G6'], ['F16', 'G1', 'E6'], ['F15', 'F34', 'F26'], ['G4', 'E1', 'F14'], ['F13', 'F24', 'F56'], ['G2', 'F25', 'E5'], ['F35', 'G5', 'E3'], ['E2', 'G3', 'F23']]

[['F14', 'E1', 'G4'], ['G3', 'F23', 'E2'], ['G5', 'F35', 'E3'], ['F46', 'E4', 'G6'], ['G2', 'F25', 'E5'], ['F16', 'E6', 'G1'], ['F36', 'F45', 'F12'], ['F56', 'F13', 'F24'], ['F26', 'F34', 'F15']]

[['E1', 'G4', 'F14'], ['F34', 'F15', 'F26'], ['E2', 'G3', 'F23'], ['F36', 'F12', 'F45'], ['F46', 'G6', 'E4'], ['E5', 'F25', 'G2'], ['F16', 'G1', 'E6'], ['G5', 'E3', 'F35'], ['F13', 'F56', 'F24']]



In [37]:
gens_lines = [lines_perms[index] for index in gens_indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('E4', 'G6', 'E3', 'G5', 'E5', 'E2'),
  ('G2', 'E5', 'F15', 'F26', 'F56', 'F45'),
  ('G4', 'F14', 'F23', 'E2', 'F35', 'F25'),
  ('E1', 'G4', 'E3', 'G5', 'F24', 'E6')],
 [(1,4,6)(2,3,5)(7,10,12)(8,9,11)(13,22,27)(14,25,21)(15,26,17)(16,19,24)(18,23,20),
  (1,8,13)(2,16,4)(3,27,17)(5,25,10)(6,23,14)(7,11,19)(9,21,26)(12,18,22)(15,20,24),
  (1,10,15)(2,18,9)(3,23,11)(4,12,26)(5,20,8)(6,7,17)(13,25,24)(14,19,27)(16,22,21),
  (2,3,19,22)(4,20,7,24)(5,6,25,26)(8,17,13,12)(9,11,14,16)(18,23,27,21)])